In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
import scipy.stats as sts

import random
import string

import pyspike as spk
import itertools as itl

import plotly.express as px

In [ ]:
from nb_vars import FOLDER_OUTPUT_BASE, CUTOFF_SPIKES_HZ

# Load df of filtered peaks

In [ ]:
CONDITION_NAME = "pulse-duration"
CONDITION = 500

folder_input =  f'{FOLDER_OUTPUT_BASE}/{CONDITION_NAME}_{CONDITION}'

In [ ]:
df_peaks = pd.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv')

In [ ]:
df_peaks

# Making synchrony DFs

We are going to make two different types of synchrony index dataframes: one representing pair-to-pair indices, and one with the indices per well. The first one will be useful to find similarities between electrodes and to be able to make a similarity graph; and the second one is more representative of the general elements of the well as a whole. As for the similarity metrics, we are going to use different metrics, including the ISI distance, SPIKE distance, Rate Independent SPIKE distance and SPIKE synchrony. All measurements will be adjusted using MRTS (optional, but included).

In [ ]:
list_wells = df_peaks.drop_duplicates('well')['well'].values

MRTS = 'auto'

In [ ]:
df_peaks_dedup = df_peaks.drop_duplicates(subset=['condition', 'treatment', 'well']).sort_values(by=['well', 'condition', 'treatment']).reset_index(drop=True)[['condition', 'treatment', 'well']]
df_peaks_dedup

In [ ]:
# Make DF with recompiled info per well
df_synchrony_well = pd.DataFrame(columns=['condition', 'treatment', 'well', 'ISI distance', 'SPIKE profile', 'RI SPIKE profile', 'SPIKE sync'],)

for col in ['condition', 'treatment', 'well']:
    df_synchrony_well.loc[:, col] = df_peaks_dedup.loc[:, col].values


for idx in range(len(df_peaks_dedup)):
    condition, treatment, well = df_peaks_dedup.iloc[idx].values.tolist()

    electrodes = df_peaks[df_peaks['well'] == well].drop_duplicates('electrode')['electrode'].values

    spike_trains = [spk.SpikeTrain(df_peaks[(df_peaks['electrode'] == electrode) & (df_peaks['well'] == well) & (df_peaks['treatment'] == treatment) & (df_peaks['condition'] == condition)]['time'].values, 
                                   edges=np.max(df_peaks['time'].values)) for electrode in electrodes]

    df_synchrony_well.loc[idx, 'ISI distance'] = spk.isi_distance(spike_trains, MRTS=MRTS)
    df_synchrony_well.loc[idx, 'SPIKE profile'] = spk.spike_distance(spike_trains, MRTS=MRTS)
    df_synchrony_well.loc[idx, 'RI SPIKE profile'] = spk.spike_distance(spike_trains, MRTS=MRTS, RI=True)
    df_synchrony_well.loc[idx, 'SPIKE sync'] = spk.spike_sync(spike_trains, MRTS=MRTS)

In [ ]:
# Make DF with recompiled info per electrode pair
df_synchrony_pairs = pd.DataFrame(columns=['condition', 'treatment', 'well', 'electrode_A', 'electrode_B', 'ISI distance', 'SPIKE profile', 'RI SPIKE profile', 'SPIKE sync'])

idx = 0
for ctw in range(len(df_peaks_dedup)):
    condition, treatment, well = df_peaks_dedup.iloc[ctw].values.tolist()

    electrodes = df_peaks[(df_peaks['well'] == well) & (df_peaks['treatment'] == treatment) & (df_peaks['condition'] == condition)].drop_duplicates('electrode')['electrode'].values
    for el_A, el_B in itl.combinations(electrodes, 2):
        df_synchrony_pairs.loc[idx, ['condition', 'treatment', 'well', 'electrode_A', 'electrode_B']] = [condition, treatment, well, el_A, el_B]
        
        spike_trains = [spk.SpikeTrain(df_peaks[(df_peaks['electrode'] == electrode) & (df_peaks['well'] == well) & (df_peaks['treatment'] == treatment) & (df_peaks['condition'] == condition)]['time'].values, 
                                       edges=np.max(df_peaks['time'].values)) for electrode in [el_A, el_B]]

        df_synchrony_pairs.loc[idx, 'ISI distance'] = spk.isi_distance(spike_trains, MRTS=MRTS)
        df_synchrony_pairs.loc[idx, 'SPIKE profile'] = spk.spike_distance(spike_trains, MRTS=MRTS)
        df_synchrony_pairs.loc[idx, 'RI SPIKE profile'] = spk.spike_distance(spike_trains, MRTS=MRTS, RI=True)
        df_synchrony_pairs.loc[idx, 'SPIKE sync'] = spk.spike_sync(spike_trains, MRTS=MRTS)

        idx += 1 

# Plots

## Plot well information

In [ ]:
profile_to_plot = 'SPIKE sync'

fig, axs = plt.subplots(2, 2, figsize=(14, 8))
for idx, profile_to_plot in enumerate(['ISI distance', 'SPIKE profile', 'RI SPIKE profile', 'SPIKE sync']):
    sns.barplot(data=df_synchrony_well, x='well', y=profile_to_plot, hue='treatment', ax=axs.ravel()[idx])

In [ ]:
df_synchrony_well_diff = df_synchrony_well.drop_duplicates(subset=['well']) 
df_synchrony_well_diff.loc[:, ['ISI distance', 'SPIKE profile', 'RI SPIKE profile', 'SPIKE sync']] = df_synchrony_well.groupby('well')[['ISI distance', 'SPIKE profile', 'RI SPIKE profile', 'SPIKE sync']].diff().dropna().values

In [ ]:
df_synchrony_well_diff_melt = df_synchrony_well_diff.melt(id_vars=['condition', 'treatment', 'well'])
sns.swarmplot(df_synchrony_well_diff_melt, x='variable', y='value')
plt.plot([-0.5, 3.5], [0, 0], c='red')

In [ ]:
sns.pairplot(df_synchrony_well, corner=True, hue='treatment')

## Plot pairwise correlations

In [ ]:
profile_to_plot = 'ISI distance'

list_all_wells = [f'{letter}{number}' for letter in ['A', 'B', 'C', 'D'] for number in range(1, 7)]
list_treatments = ['PRE', 'POST']

fig, axs = plt.subplots(4, 6 * 2, figsize = (6 * 2 * 3, 4 * 3))

for idx, (well, treatment) in enumerate(itl.product(list_all_wells, list_treatments)):
    df_synchrony_pairs_well = df_synchrony_pairs[(df_synchrony_pairs['well'] == well) & (df_synchrony_pairs['treatment'] == treatment)]

    if len(df_synchrony_pairs_well) > 0:
        df_sub = df_synchrony_pairs_well[['electrode_A', 'electrode_B', profile_to_plot]].rename(columns={profile_to_plot: 'value'})

        df_pivot = df_sub.pivot(index='electrode_A', columns='electrode_B', values='value').astype(float)

        sns.heatmap(df_pivot, ax=axs.ravel()[idx], vmin=df_synchrony_pairs[profile_to_plot].min(), vmax=df_synchrony_pairs[profile_to_plot].max())
        axs.ravel()[idx].set_title(f'{well} {treatment}')
    else:
        axs.ravel()[idx].set_axis_off()

    

plt.tight_layout()

In [ ]:
profile_to_plot = 'SPIKE sync'

list_all_wells = [f'{letter}{number}' for letter in ['A', 'B', 'C', 'D'] for number in range(1, 7)]
list_treatments = ['PRE', 'POST']

fig, axs = plt.subplots(4, 6, figsize = (6  * 3, 4 * 3))

for idx, well in enumerate(list_all_wells):
    df_synchrony_pairs_well_PRE = df_synchrony_pairs[(df_synchrony_pairs['well'] == well) & (df_synchrony_pairs['treatment'] == 'PRE')]
    df_synchrony_pairs_well_POST = df_synchrony_pairs[(df_synchrony_pairs['well'] == well) & (df_synchrony_pairs['treatment'] == 'POST')]

    if len(df_synchrony_pairs_well_POST) > 0:
        df_sub_PRE = df_synchrony_pairs_well_PRE[['electrode_A', 'electrode_B', profile_to_plot]].rename(columns={profile_to_plot: 'value'})
        df_pivot_PRE = df_sub_PRE.pivot(index='electrode_A', columns='electrode_B', values='value').astype(float)

        df_sub_POST = df_synchrony_pairs_well_POST[['electrode_A', 'electrode_B', profile_to_plot]].rename(columns={profile_to_plot: 'value'})
        df_pivot_POST = df_sub_POST.pivot(index='electrode_A', columns='electrode_B', values='value').astype(float)

        sns.heatmap(df_pivot_PRE - df_pivot_POST, ax=axs.ravel()[idx], vmin=-0.1, vmax=0.1, cmap='coolwarm')
        axs.ravel()[idx].set_title(f'{well} PRE - POST')
    else:
        axs.ravel()[idx].set_axis_off()

    

plt.tight_layout()

In [ ]:
well = 'D3'

In [ ]:
# Plot each well
df_peaks_sub = df_peaks[(df_peaks['treatment'] == 'PRE') & (df_peaks['well'] == well)]

y, x = [(int(i[0]) - 1) * 4 + (int(i[1]) - 1) for i in df_peaks_sub['electrode'].astype(str).values], df_peaks_sub['time'].values 

fig = px.scatter(x=x, y=y, opacity=0.35)
fig.update_yaxes(tickvals=np.arange(16), ticktext=[f'{i//4 + 1}{i%4 + 1}' for i in np.arange(16)])

In [ ]:
# Plot each well
df_peaks_sub = df_peaks[(df_peaks['treatment'] == 'POST') & (df_peaks['well'] == well)]

y, x = [(int(i[0]) - 1) * 4 + (int(i[1]) - 1) for i in df_peaks_sub['electrode'].astype(str).values], df_peaks_sub['time'].values 

fig = px.scatter(x=x, y=y, opacity=0.35)
fig.update_yaxes(tickvals=np.arange(16), ticktext=[f'{i//4 + 1}{i%4 + 1}' for i in np.arange(16)])